In [ ]:
# Cofiguration
# from elevation_aware_ssl.core import config
# config.config_notebook(False)

# Load libraries
from typing import List, Callable, Union, Tuple
import multiprocessing as mp
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import os
import matplotlib.patches as mpatches
import numpy as np
import torch
from tqdm.autonotebook import tqdm

# custom libary
import elevation_aware_ssl.preprocessing as DP
from elevation_aware_ssl.EDA import image_label_sanity_check
from elevation_aware_ssl import EDA
from elevation_aware_ssl.regression.metrics import utils

# from elevation_aware_ssl.utils import metrics, simclr, data_for_seg
# from elevation_aware_ssl.utils import data_for_seg

import wandb
wandb.login(key="wandb_key")

In [ ]:
drive_path = "/media/omar/storage/gdrive"

In [ ]:
path_to_label = f"{drive_path}/Maestria/Datasets/GeoDataset/" 
select_classes = ['non_agricultural_area', 'legal_exclusions', 'agricultural_frontier']

In [ ]:
#Load metadata
path_to_metadata =  f'{drive_path}/Maestria/Datasets/GeoDataset/metadata/metadata_binary.csv'
metadata = pd.read_csv(path_to_metadata).sample(frac=1, random_state=42)
print(metadata.shape)
metadata.head()

In [ ]:
project = "Regression"
entity = "omar_castano"
name = "report_glcnet"
version = [
    "RandomInit",
    "RandomInit",
    "RandomInit",
    "SimCLR",
    "SimCLR",
    "SimCLR",
]
train_size = [
    0.005,
    0.01,
    0.05,
    0.005,
    0.01,
    0.05,
]


bar_plots = {}
for metric in ["test_RMSE"]:
    plot = utils.barplot_metrics_from_wandb(
        wandb,
        project=project,
        entity=entity,
        version=list(set(version)),
        train_size=list(set(train_size)),
        metric=f"{metric}",
    )

    bar_plots.update({metric: plot})

In [ ]:
bar_plots["test_RMSE"]

In [ ]:
def standar_report(wandb, project, entity, name, version, train_size):

    bar_plots = {}
    for metric in ["test_RMSE"]:
        plot = utils.barplot_metrics_from_wandb(
            wandb,
            project=project,
            entity=entity,
            version=list(set(version)),
            train_size=list(set(train_size)),
            metric=f"{metric}",
        )

        bar_plots.update({metric: plot})

   
    loss_table = utils.get_table(
        wandb,
        project=project,
        entity=entity,
        version=version,
        train_size=train_size,
        table_name="Loss",
    )

    loss_plot = utils.plot_loss_curves(loss_table, color="train_size", line_dash="version")

    with wandb.init(
        project=project,
        entity=entity,
        config={"version": "report", "train_size": None},
        name=name,
    ) as run:

        [run.log({f"Bareplot {metric}": plot}) for metric, plot in bar_plots.items()]
        run.log({"Loss Plots": loss_plot})

In [ ]:
standar_report(
    wandb,
    project="Regression",
    entity="omar_castano",
    name="report",
    version=[
        "RandomInit",
        "RandomInit",
        "RandomInit",
        # "RandomInit",
        "SimCLR",
        "SimCLR",
        "SimCLR",
        # "SimCLR",
    ],
    train_size=[
        0.005,
        0.01,
        0.05,
        # 0.1,
        0.005,
        0.01,
        0.05,
        # 0.1,
    ],
)